In [1]:
import os
import glob
import psycopg2
import pandas as pd
import numpy as np
import json
import sql_queries
from typing import List

Connecting to the database

In [2]:
with open("../config.json") as f:
    config = json.load(f)
    
conn = psycopg2.connect(host = config["host"],
                        dbname = config["dbname"],
                        user = config["username"],
                        password = config["password"])

conn.set_session(autocommit=True)
cur = conn.cursor()

# Order of table filling
Due to the foreign Key constraints, we will need to insert the data in a certain order
* Artists need to be inserted before songs
* All dimension tables need to be filled before the fact table

An order that satisfied these constraints can be:
1. Artist table
2. Song table
3. User table 
4. Time table
5. Songplay table

## 1. The artist table

In [3]:
df_artist = pd.read_csv("../data/cleaned/artists.csv")
df_artist.head()

,artist_id,artist_name,artist_location,artist_longitude,artist_latitude
0,ARD7TVE1187B99BFB1,Casual,California - LA,NaN,NaN
1,ARMJAGH1187FB546F3,The Box Tops,"Memphis, TN",-90.04892,35.14968
2,ARKRRTF1187B9984DA,Sonora Santanera,NaN,NaN,NaN
3,AR7G5I41187FB4CE6C,Adam Ant,"London, England",NaN,NaN
4,ARXR32B1187FB57099,Gob,NaN,NaN,NaN


* It's important to note that np.nan is a float value, and doesn't automatically translate to `NULL` in the database. 

For example,

In [4]:
cur.execute(sql_queries.artist_table_insert, list(df_artist.iloc[2].values))

In [5]:
cur.execute("SELECT * FROM artist")
res = cur.fetchone()
print(res)
print(type(res[-1]))

('ARD7TVE1187B99BFB1', 'Casual', 'California - LA', None, None)
<class 'NoneType'>


* So, the values stored in the database are not `NULL`. 
* This can cause confusion when we later fetch data from the database. 
* We should standardize the way we represent NULLs

In [6]:
df_artist.fillna(psycopg2.extensions.AsIs("NULL"), inplace = True)

In [7]:
cur.execute("DELETE FROM artist")
cur.execute(sql_queries.artist_table_insert, list(df_artist.iloc[2].values))

ForeignKeyViolation: update or delete on table "artist" violates foreign key constraint "song_artist_id_fkey" on table "song"
DETAIL:  Key (artist_id)=(ARD7TVE1187B99BFB1) is still referenced from table "song".


In [8]:
cur.execute("SELECT * FROM artist")
res = cur.fetchone()
print(res)

('ARD7TVE1187B99BFB1', 'Casual', 'California - LA', None, None)


## 2. Songs table

In [9]:
df_songs = pd.read_csv("../data/cleaned/songs.csv")
df_songs.head()

,song_id,title,artist_id,duration,year
0,SOMZWCG12A8C13C480,I Didn't Mean To,ARD7TVE1187B99BFB1,218.93179,NaN
1,SOCIWDW12A8C13D406,Soul Deep,ARMJAGH1187FB546F3,148.03546,1969.0
2,SOXVLOJ12AB0189215,Amor De Cabaret,ARKRRTF1187B9984DA,177.47546,NaN
3,SONHOTT12A8C13493C,Something Girls,AR7G5I41187FB4CE6C,233.40363,1982.0
4,SOFSOCN12A8C143F5D,Face the Ashes,ARXR32B1187FB57099,209.60608,2007.0


In [10]:
df_songs.fillna(psycopg2.extensions.AsIs("NULL"), inplace = True)

In [11]:
cur.execute(sql_queries.song_table_insert, list(df_songs.iloc[2].values))

UniqueViolation: duplicate key value violates unique constraint "song_pkey"
DETAIL:  Key (song_id)=(SOXVLOJ12AB0189215) already exists.


In [12]:
cur.execute("SELECT * FROM song")
res = cur.fetchone()
print(res)

('SOMZWCG12A8C13C480', "I Didn't Mean To", 'ARD7TVE1187B99BFB1', 219, None)


## 3. The time table

In [13]:
df_times = pd.read_csv("../data/cleaned/time.csv")
df_times.head()

,ts,hour,day,weekday,week,month,year
0,1541105830796,20,1,3,44,11,2018
1,1541106106796,21,1,3,44,11,2018
2,1541106132796,21,1,3,44,11,2018
3,1541106352796,21,1,3,44,11,2018
4,1541106496796,21,1,3,44,11,2018


In [14]:
df_times.fillna(psycopg2.extensions.AsIs("NULL"), inplace = True)

In [15]:
cur.execute(sql_queries.time_table_insert, list(df_times.iloc[0].values))

ProgrammingError: can't adapt type 'numpy.int64'

* It looks like psycopg2 doesn't understand how to convert `np.int64` types to sql.
* As such, we will have to use something called `adapter`. It's a psycopg2 object through which we can tell pycopg2 how to translate certain values to SQL.

I learned that from this [stackoverflow post](https://stackoverflow.com/questions/50626058/psycopg2-cant-adapt-type-numpy-int64)

In [ ]:
from psycopg2.extensions import register_adapter
def adapt_np_int(np_int:np.int64):
    _INT = psycopg2.extensions.Int
    return _INT(np_int)
    
register_adapter(np.int64, adapt_np_int)

In [16]:
cur.execute(sql_queries.time_table_insert, list(df_times.iloc[0].values))

ProgrammingError: can't adapt type 'numpy.int64'

In [17]:
cur.execute("SELECT * FROM time_table")
res = cur.fetchone()
print(res)

(1541105830796, 20, 1, 3, 44, 11, 2018)


## 4. User Data

In [18]:
df_users = pd.read_csv("../data/cleaned/users.csv")
df_users.head()

,userId,firstName,lastName,gender,level
0,2.0,Jizelle,Benjamin,F,free
1,3.0,Isaac,Valdez,M,free
2,4.0,Alivia,Terrell,F,free
3,5.0,Elijah,Davis,M,free
4,6.0,Cecilia,Owens,F,free


In [19]:
df_users.fillna(psycopg2.extensions.AsIs("NULL"), inplace = True)

In [20]:
cur.execute(sql_queries.user_table_insert, list(df_users.iloc[0].values))

In [21]:
cur.execute("SELECT * FROM user_table")
res = cur.fetchone()
print(res)

(3, 'Isaac', 'Valdez', 'M', 'free')


## 5. The songplay table

In [22]:
df_songplays = pd.read_csv("../data/cleaned/songplays.csv")
df_songplays.head()

,sessionId,ts,song,length,artist,userId,level,location,userAgent
0,139,1541106106796,You Gotta Be,246.30812,Des'ree,8.0,free,"Phoenix-Mesa-Scottsdale, AZ","""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK..."
1,139,1541106352796,Flat 55,144.03873,Mr Oizo,8.0,free,"Phoenix-Mesa-Scottsdale, AZ","""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK..."
2,139,1541106496796,Quem Quiser Encontrar O Amor,177.18812,Tamba Trio,8.0,free,"Phoenix-Mesa-Scottsdale, AZ","""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK..."
3,139,1541106673796,Eriatarka,380.42077,The Mars Volta,8.0,free,"Phoenix-Mesa-Scottsdale, AZ","""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK..."
4,139,1541107053796,Becoming Insane,440.26730,Infected Mushroom,8.0,free,"Phoenix-Mesa-Scottsdale, AZ","""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK..."


In [23]:
for index, row in df_songplays.iterrows():

    # get songid and artistid from song and artist tables
    cur.execute(sql_queries.song_select, (row.song, row.artist, row.length))
    results = cur.fetchone()
    
    if results:
        songid, artistid = results
    else:
        songid, artistid = None, None

    #print((row["ts"], row["userId"], row["level"], songid, artistid, row["sessionId"], row["location"], row["userAgent"]))
    # insert songplay record
    songplay_data = (row["ts"], row["userId"], row["level"], songid, artistid, row["sessionId"], row["location"], row["userAgent"])
    cur.execute(sql_queries.songplay_table_insert, songplay_data)

So, that's a violation of the foreign key constraint, because we haven't filled the dimension tables yet. That'll be done in the script.

That's it for this notebook!

N.B: I would say we could do our joins only once in pandas instead of doing table joins (`song_select`) for EACH entry in the songplay table. That would be more efficient.

In [28]:
songs_artists = df_songs.merge(df_artist, on = "artist_id")[["title", "artist_name", "song_id", "artist_id", "duration"]]
songs_artists.head()

,title,artist_name,song_id,artist_id,duration
0,I Didn't Mean To,Casual,SOMZWCG12A8C13C480,ARD7TVE1187B99BFB1,218.93179
1,OAKtown,Casual,SOQLGFP12A58A7800E,ARD7TVE1187B99BFB1,259.44771
2,Soul Deep,The Box Tops,SOCIWDW12A8C13D406,ARMJAGH1187FB546F3,148.03546
3,Amor De Cabaret,Sonora Santanera,SOXVLOJ12AB0189215,ARKRRTF1187B9984DA,177.47546
4,Something Girls,Adam Ant,SONHOTT12A8C13493C,AR7G5I41187FB4CE6C,233.40363


In [29]:
merged = df_songplays.merge(songs_artists, left_on = ["song", "artist", "length"], right_on = ["title", "artist_name", "duration"], how = "left")
merged 

,sessionId,ts,song,length,artist,userId,level,location,userAgent,title,artist_name,song_id,artist_id,duration
0,139,1541106106796,You Gotta Be,246.30812,Des'ree,8.0,free,"Phoenix-Mesa-Scottsdale, AZ","""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",NaN,NaN,NaN,NaN,NaN
1,139,1541106352796,Flat 55,144.03873,Mr Oizo,8.0,free,"Phoenix-Mesa-Scottsdale, AZ","""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",NaN,NaN,NaN,NaN,NaN
2,139,1541106496796,Quem Quiser Encontrar O Amor,177.18812,Tamba Trio,8.0,free,"Phoenix-Mesa-Scottsdale, AZ","""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",NaN,NaN,NaN,NaN,NaN
3,139,1541106673796,Eriatarka,380.42077,The Mars Volta,8.0,free,"Phoenix-Mesa-Scottsdale, AZ","""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",NaN,NaN,NaN,NaN,NaN
4,139,1541107053796,Becoming Insane,440.26730,Infected Mushroom,8.0,free,"Phoenix-Mesa-Scottsdale, AZ","""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6815,1076,1543603205796,The Pretender,271.38567,Foo Fighters,16.0,paid,"Birmingham-Hoover, AL","""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",NaN,NaN,NaN,NaN,NaN
6816,1076,1543603476796,Besos De Ceniza,202.60526,Timbiriche,16.0,paid,"Birmingham-Hoover, AL","""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",NaN,NaN,NaN,NaN,NaN
6817,1076,1543603678796,Rose,206.05342,A Perfect Circle,16.0,paid,"Birmingham-Hoover, AL","""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",NaN,NaN,NaN,NaN,NaN
6818,1076,1543603884796,The Haunting,348.68200,Anberlin,16.0,paid,"Birmingham-Hoover, AL","""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",NaN,NaN,NaN,NaN,NaN


In [26]:
merged[["ts", "userId", "level", "song_id", "artist_id", "sessionId", "location", "userAgent"]]

,ts,userId,level,song_id,artist_id,sessionId,location,userAgent
0,1541106106796,8.0,free,NaN,NaN,139,"Phoenix-Mesa-Scottsdale, AZ","""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK..."
1,1541106352796,8.0,free,NaN,NaN,139,"Phoenix-Mesa-Scottsdale, AZ","""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK..."
2,1541106496796,8.0,free,NaN,NaN,139,"Phoenix-Mesa-Scottsdale, AZ","""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK..."
3,1541106673796,8.0,free,NaN,NaN,139,"Phoenix-Mesa-Scottsdale, AZ","""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK..."
4,1541107053796,8.0,free,NaN,NaN,139,"Phoenix-Mesa-Scottsdale, AZ","""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK..."
...,...,...,...,...,...,...,...,...
6815,1543603205796,16.0,paid,NaN,NaN,1076,"Birmingham-Hoover, AL","""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4..."
6816,1543603476796,16.0,paid,NaN,NaN,1076,"Birmingham-Hoover, AL","""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4..."
6817,1543603678796,16.0,paid,NaN,NaN,1076,"Birmingham-Hoover, AL","""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4..."
6818,1543603884796,16.0,paid,NaN,NaN,1076,"Birmingham-Hoover, AL","""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4..."


And we would use this resulting dataframe directly.